In [5]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [ ]:
dataset_path = "/content/drive/MyDrive/animals"
animals = os.listdir(dataset_path)
print(animals)

In [ ]:
selected_images = {}
for animal in animals:
  animal_path = os.path.join(dataset_path, animal)
  images = os.listdir(animal_path)
  selected_images[animal] = images

In [ ]:
image_size = 32
images_count = 100
k_values = list(range(2, 20))

In [ ]:
train_imgs = []

valid_extensions = ('.jpg', '.jpeg', '.png', '.bmp')

for index, animal in enumerate(animals):
    label = index
    path = os.path.join(dataset_path, animal)
    images = sorted(os.listdir(path))  # deterministic order

    for img in images[:images_count]:
        if not img.lower().endswith(valid_extensions):
            continue

        img_path = os.path.join(path, img)
        img_bgr = cv2.imread(img_path)
        if img_bgr is None:
            print(f"Warning: Failed to load image: {img_path}")
            continue

        img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
        img_gray = cv2.resize(img_gray, (image_size, image_size))
        train_imgs.append([img_gray, label])

# Shuffle with fixed seed for consistent order every run
np.random.seed(42)
np.random.shuffle(train_imgs)

In [ ]:
rows, cols = 5, 5
plt.figure(figsize=(30, 30))

for i in range(rows * cols):
    img, label = train_imgs[i]
    plt.subplot(rows, cols, i + 1)
    plt.imshow(img, cmap='gray')
    plt.title(label, fontsize=10)
    plt.axis('off')

plt.show()

In [ ]:
def l1_distance(a, b):
    return np.sqrt(np.sum((a - b) ** 2))

In [ ]:
def predict(X_train, y_train, x_test, k, dist_func):
    distances = [dist_func(x_test, x) for x in X_train]
    nearest = np.argsort(distances)[:k]
    nearest_labels = y_train[nearest]
    values, counts = np.unique(nearest_labels, return_counts=True)
    return values[np.argmax(counts)]

In [ ]:
def cross_validation(X, Y, k_values, folds=5):
    indices = np.arange(len(X))
    np.random.shuffle(indices)
    fold_size = len(X) // folds
    fold_indices = [indices[i*fold_size:(i+1)*fold_size] for i in range(folds)]

    accuracy_l1 = {k: [] for k in k_values}

    for i in range(folds):
        val_idx = fold_indices[i]
        train_idx = np.hstack([fold_indices[j] for j in range(folds) if j != i])
        X_train, Y_train = X[train_idx], Y[train_idx]
        X_val, Y_val = X[val_idx], Y[val_idx]

        for k in k_values:
            correct_l1 = 0
            for x, y_true in zip(X_val, Y_val):
                pred_l1 = predict(X_train, Y_train, x, k, l1_distance)
                correct_l1 += int(pred_l1 == y_true)
            accuracy_l1[k].append(correct_l1 / len(Y_val))
        print(f"Fold {i+1} completed.")

    avg_l1 = {k: np.mean(accuracy_l1[k]) for k in k_values}
    return avg_l1

In [ ]:
X = np.array([img[0].flatten() for img in train_imgs])
Y = np.array([img[1] for img in train_imgs])

avg_l1= cross_validation(X, Y, k_values)

for k in k_values:
    print(f"Average Accuracy for k={k} (L1): {avg_l1[k]:.2f}")

In [ ]:
plt.figure(figsize=(10, 6))

plt.plot(k_values, [avg_l1[k] for k in k_values], marker='s', label='L1 (Euclidean)', color='green')

plt.xlabel("K")
plt.ylabel("Average Accuracy")
plt.xticks(k_values)  # Ensure integer ticks on x-axis
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
best_k_l1 = max(avg_l1, key=avg_l1.get)

Top 5 Predictions:

In [ ]:
test_indices = random.sample(range(len(X)), 5)
X_test = X[test_indices]
Y_test = Y[test_indices]

for i, (x, true_label) in enumerate(zip(X_test, Y_test)):
    pred_l1 = predict(X, Y, x, k=best_k_l1, dist_func=l1_distance)


    fig, ax = plt.subplots()
    ax.imshow(x.reshape(32, 32), cmap='gray')
    ax.axis('off')
    ax.set_title(
        f"True: {true_label}\n"
        f"L1 Prediction: {pred_l1} (k={best_k_l1})"

    )
    plt.show()